### Imports

In [250]:
import pandas as pd
import requests
import diffbot
import string

In [285]:
from tweepy import OAuthHandler
from googletrans import Translator
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import re

In [158]:
consumer_key="4BqJe9MamSVRkRXiPFlcjTxqa"
consumer_secret="TgGbjV3fAbrDJOnDQM9iD07d9rjOnkVK2N4nMiqcN3UJI8phwC"
access_token_key="871991040256999424-owsFNOcGYr7btQVKpeZDGBeET0347ot"
access_token_secret="lx7Vv0SHGICxrcxSR5mXAnOW4w26DhEcQZ1R99d1W2EIB"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = API(auth, wait_on_rate_limit=True)

### Translation

In [283]:
def trans_tweet(tweet):
    tra = Translator()
    return tra.translate(tweet, dest="en").text

### Preprocessing

In [277]:
def rmv_par(text):
    text = text.replace("«","")
    text = text.replace("»","")
    text = text.replace('"',"")
    text = text.replace("'","")
    return text

In [265]:
def rmurl(text):
    '''
    This function removes all the URLs, the #hashtag and the @user of a column made of strings.
    Be careful to apply it BEFORE all the other preprocessing steps (if not it wont'
    be recognized as a URL)
    '''
    result = re.sub(r"http\S+|www.\S+|@\S+|#\S+", "", text)
    #text = text.replace('http\S+|www.\S+|@\S+|#\S+', '')
    return result

In [266]:
# Lowercase the tweet's column
def lower(text):
    '''
    This function lowercases a column made of strings.
    '''
    text = text.lower()
    return text

In [245]:
# Remove the numbers in the tweet's column
def remove_numbers(text):
    return ''.join(word for word in text if not word.isdigit())
    

In [251]:
def replace_punct(text):
    punct = string.punctuation
    for punctu in punct:
        text = text.replace(punctu, ' ')
        text = text.replace(' rt ','')
        text = " ".join(text.split())
    return text

In [256]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [333]:
def delete_paragraph(paragraph):
    without_line_breaks = a_string.replace("\n", " ")
    return without_line_breaks

In [279]:
def preprocessing (text):
    text = rmv_par(text)
    text = rmurl(text)
    text = lower(text)
    text = remove_numbers(text)
    text = replace_punct(text)
    text = deEmojify(text)
    return text

### Functions

In [286]:
def text_from_url(url):
    url = input("Write a Url: ")
    new_url = f'https://extractorapi.com/api/v1/extractor/?apikey=f6d8320079329a09fedc7c3c59cc581740f904d3&url={url}'
    response = requests.get(new_url)
    text = response.json()["text"]
    text = trans_tweet(text)
    text = preprocessing(text)
    return text

In [338]:
text_from_url("https://www.politico.eu/article/while-brussels-starts-plenary-strasbourg-suffers-from-lack-of-meps/")

Write a Url: https://www.politico.eu/article/while-brussels-starts-plenary-strasbourg-suffers-from-lack-of-meps/


'peer review looks to counter moscows effort to downplay accusations it was behind russian opposition leaders poisoning the islamic republic is still looking to avenge the death of qassem soleimani officials said the eescs jacek krawczyk hits out at his boss and at olaf the eus anti fraud unit former interior minister will have to lead reform efforts amid internal party divisions the most damaging revelations about this president come not from shadowy bureaucrats but from the people closest to him decision sanctions her lack of actions and her acceptance of the ongoing crimes against the rohingya community in myanmar political calculus that created dangerous conditions at refugee camp will survive the fire us secretary of state said efforts to poison dissidents put black marks on countries and he condemned the poisoning as not how normal countries operate economic ties have caused berlin to tread lightly on human rights brussels faces demands for quick action to help thousands displace

In [336]:
def text_from_tweets(username):
    all_tweets = []
    content = []
    content_list = []
    for tweet in Cursor(api.user_timeline,id=username,tweet_mode='extended').items(100):
        if "retweeted_status" in dir(tweet):
            tweet=tweet.retweeted_status.full_text
            content_list.append(tweet)
        else:
            tweet=tweet.full_text
            content_list.append(tweet)
            content.append(len(content_list))
    all_tweets.append(content_list)
    new_tweets = []
    for i in all_tweets[0]: 
        text = trans_tweet(i)
        text = preprocessing(i)
        new_tweets.append(text)
    return new_tweets

In [337]:
text_from_tweets("the_simonpastor")

['louvres paris',
 '',
 'theres nothing like le tour de france ',
 'huge twitter project coming up in two weeks stay tuned youll love it ',
 'coolest civic gov techs you know',
 'months ago i wrote about how covid could affect social interactions which i argued could result in a community come back increased loneliness and a questioning of ones social circles what do you think or is it still too early to tell',
 'is my new favorite cooking website  it almost encourages me to start cooking ',
 'a wordcloud of obamas speech at the ',
 'saw this on linkedin today have to admit i was touched',
 'does anyone work for or knows about interesting political data companies',
 'bravo  l pour son parcours hroque et au pour sa qualification en finale de la ligue des champions dimanche ans aprs l la mythique coupe peut retrouver le sol franais toutes et tous derrire le ltoile est au bout de lexploit',
 'nous sommes fiers de vous  ',
 'champions league final ',
 'today marks the second anniversary of

In [335]:
def simple_text(text):
    text = input("Write something: ")
    text = delete_paragraph(text)
    text = trans_tweet(text)
    text = preprocessing(text)
    return text

Supported News Sites for URL Extractor:
French: Le Monde, Le Figaro, Libération, Ouest France, L'Opinion, Les Echos, Le Parisien, Marianne, Valeurs Actuelles
European : Politico.eu
British : The Guardian, The Daily Mail, The Telegraph, The Independent, Daily Mirror, The Sun, The Times, Evening Standard
Beligan : Het Nieuwsblad, Le Soir
Spanish : El Mundo, La Vanguardia
Portuguese : Jornal de Noticias, Jornal Economico
Italian : La Repubblica, Corriere Della Sera, La Stampa
Dutch : De Telegraaf, Die Presse
German : Bild, Die Welt, Süddeutsche Zeitung,Frankfurter Allgemeine Zeitung
Austrian : Der Standard, 
Czech : Mladá fronta DNES, Blesk
Polish : Gazeta Wyborcza, 
Swedish : Expressen, Svenska Dagbladet
Slovakian : SME, 
Bosnia : Dnevni avaz, 
Malta : The Times of Malta
Swiss : Blick
Norwegian : Aftenposten
Russian : Komsomolskaya Pravda, Rossiyskaya Gazeta, Moskovskij Komsomolets, Izvetsia, Russia Today, Sputnik News
China : China Daily
American : Five Thirty-Eight, CNN, Fox News, Breitbart, NBC News, CBS News, Politico, Vice, Wired, The Washington Post, The Los Angeles Times
Global : Al-Jazeera, BBC, Reuters, TechCrunch
Unsupported: New York Times, Financial Times, El Pais, Die Zelt, Kommersant, Verdens Gang, Neue Zürcher Zeitung, Dagen Nyheter, Volkstkrant, Der Spiegel, Huffington Post, NPR, ABC, Slate, Wall Street Journal, 